In [ ]:
import arcticdb as adb
import json
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import dtale
from sklearn.decomposition import PCA
from tradingo import backtest
from tradingo.utils import get_instruments, get_config, with_instrument_details

In [ ]:
import logging
logging.getLogger("tradingo.backtest").setLevel(logging.INFO)

In [ ]:
a = adb.Arctic("lmdb:///home/rory/dev/airflow/test/arctic.db")
signals = a.get_library("MODEL_SIGNALS")
backtests = a.get_library("BACKTEST")
prices = a.get_library("ASSET_PRICES")
portfolios = a.get_library("PORTFOLIO")

In [ ]:
instruments = get_instruments(get_config())

orig_prices = prices.read("ETFT.ADJ_CLOSE.yfinance")
start_dates = orig_prices.data.apply(lambda i: i.first_valid_index()).rename("start_date")
instruments = instruments.merge(start_dates, left_index=True, right_index=True, how='left')

In [ ]:
PRICES = with_instrument_details((orig_prices.data[
                instruments[(instruments.start_date < pd.Timestamp("2018-03-01"))].index
          ]).dropna(), instruments, columns=["Asset Class"])
RETURNS = np.log(PRICES/PRICES.shift()).dropna()
ORIG_PRICES = prices.read("ETFT.CLOSE.yfinance").data

In [ ]:
prices.list_symbols()

In [ ]:
pf = portfolios.read('ETFT.trend.RAW.SHARES').data
pxs = prices.read('ETFT.CLOSE.yfinance').data
bt = backtest.backtest(
    prices=pxs,
    portfolio=pf,
    start_date=pd.Timestamp("2018-01-01 00:00:00+00:00"),
    end_date=pd.Timestamp.now("utc"),
    name="trend",
    stage="RAW",
    config_name="ETFT",
    provider="yfinance",
    dry_run=True
)

In [ ]:
bt[-1]["AGG"]

In [ ]:
portfolios.read("ETFT.trend.RAW.SHARES").data.diff()
prices.read("ETFT.CLOSE.yfinance")

In [ ]:
bt[-1].loc[:,(slice(None), "total_pnl")].droplevel(1, axis=1)

In [ ]:
PRICES.transpose().groupby(["Asset Class"]).mean().transpose().plot()
(1+RETURNS.loc[:,"Commodities & Digital Assets"].transpose().merge(instruments.Name, left_index=True, right_index=True).reset_index(drop=True).set_index(["Name"]).transpose()).cumprod().plot()

In [ ]:
bt

In [ ]:
portfolios.list_symbols()

In [ ]:
pd.concat([pd.DataFrame([i.to_dict() for i in j]) for j in bt[-1]], axis=1, keys=trades.columns)